In [1]:
import os
import cv2
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import calendar
from modules.metrics import rmse
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras import backend as K
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from tensorflow.keras.layers import (
    Input, LSTM, ConvLSTM2D, Dense, Conv2D, AveragePooling2D,
    Dropout, SpatialDropout2D, MaxPooling2D, BatchNormalization,
    TimeDistributed, LeakyReLU, Flatten, Concatenate, GlobalAveragePooling2D
)
from tensorflow.keras.utils import Sequence, plot_model
from sklearn.model_selection import train_test_split

In [2]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)
print(gpus)

1 Physical GPUs, 1 Logical GPUs
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


# 資料分割

In [3]:
# split data by month
def split_data_by_month(data_path):
    train_files, val_files, test_files = [], [], []
    file_list = sorted(os.listdir(data_path))  # 20210101-160505_1_160505.csv
    train_cases_to_exclude = {'20210116', '20210530', '20210825', '20210722', '20220904'}

    for file_name in file_list:
        datetime = file_name.split('-')[0]
        year = int(datetime[0:4])
        month = int(datetime[4:6])
        day = int(datetime[6:8])
        # 取得當月的天數
        days_in_month = calendar.monthrange(year, month)[1]
        part1_end = int(days_in_month * 0.7)
        part2_end = int(days_in_month * 0.85)

        file_path = os.path.join(data_path, file_name)

        # 個案排除，加入倒測試集
        if datetime in train_cases_to_exclude:
            test_files.append(file_path)
            continue

        if day <= part1_end:
            train_files.append(file_path)
        elif day <= part2_end:
            val_files.append(file_path)
        else:
            test_files.append(file_path)

    return (train_files, val_files, test_files)



# split data by sequence
def split_data_by_sequence(data_path):
    train_files, val_files, test_files = [], [], []
    file_list = sorted(os.listdir(data_path))  # 20210101-160505_1_160505

    for file_name in file_list:
        datetime = file_name.split('-')[0]

        file_path = os.path.join(data_path, file_name)
        if datetime.startswith(('2021', '202201', '202202', '202203', '202204', '202205', '202206')):
            train_files.append(file_path)
        if datetime.startswith(('202207', '202208', '202209')):
            val_files.append(file_path)
        if datetime.startswith(('202210', '202211', '202212')):
            test_files.append(file_path)

    return (train_files, val_files, test_files)


# mont and sequence data intersect
def intersect_test_files(cells_radar_path, cells_csv_path):
    test_radar_files, test_csv_files = [], []
    file_list = sorted(os.listdir(cells_radar_path))  # 20210101-160505_1_160505.csv

    for file_name in file_list:
        datetime = file_name.split('-')[0]

        file_path = os.path.join(cells_radar_path, file_name)
        
        if not datetime.startswith(('202210', '202211', '202212')):
            continue

        datetime = file_name.split('-')[0]
        year = int(datetime[0:4])
        month = int(datetime[4:6])
        day = int(datetime[6:8])
        # 取得當月的天數
        days_in_month = calendar.monthrange(year, month)[1]
        part1_end = int(days_in_month * 0.7)
        part2_end = int(days_in_month * 0.85)
        file_path = os.path.join(cells_radar_path, file_name)

        if day > part2_end:
            test_radar_files.append(file_path)

    def make_csv_paths(radar_files):
        return [os.path.join(cells_csv_path, os.path.basename(f) + '.csv') for f in radar_files]
    test_csv_files = make_csv_paths(test_radar_files)

    return test_radar_files, test_csv_files


def get_cells_csv_files(cells_csv_path, train_radar_files, val_radar_files, test_radar_files):
    def make_csv_paths(radar_files):
        return [os.path.join(cells_csv_path, os.path.basename(f) + '.csv') for f in radar_files]

    train_cells_files = make_csv_paths(train_radar_files)
    val_cells_files = make_csv_paths(val_radar_files)
    test_cells_files = make_csv_paths(test_radar_files)

    return train_cells_files, val_cells_files, test_cells_files

# 資料生成器

In [4]:
def radar_grid_diff_processing(radar_grids):
    diff_radar_grids = []
    for i in range(len(radar_grids) - 1):
        radar_grid_diff = cv2.absdiff(radar_grids[i + 1], radar_grids[i])
        diff_radar_grids.append(radar_grid_diff)
    return np.expand_dims(np.array(diff_radar_grids), axis=-1)

def radar_csv_sliding_window_generator(radar_files, csv_files, column_name, window_size, step_size, height, width, channels, mode):
    """
    格點和CSV滑動窗口生成器，從格點資料夾中讀取格點資料，從CSV中讀取y值。
    """
    for radar_folder_path, csv_file_path in zip(radar_files, csv_files):
        # 讀取格點數據並處理差值
        radar_grids = []
        radar_grid_names = sorted(os.listdir(radar_folder_path))
        for radar_grid_name in radar_grid_names:
            radar_grid_path = os.path.join(radar_folder_path, radar_grid_name)
            radar_grid = np.load(radar_grid_path)
            radar_grid_resized = cv2.resize(radar_grid, (height, width))
            radar_grids.append(radar_grid_resized)

        diff_radar_grids = radar_grid_diff_processing(np.array(radar_grids))

        # 讀取CSV數據並計算差值
        data = pd.read_csv(csv_file_path, encoding='utf-8',
                           dtype={'fileName': str, 'day': str, 'time': str})
        data_diff = data[column_name].diff().dropna().reset_index(drop=True)


        # 提取 y 值
        lat_diff = data[column_name[0]].diff().dropna().reset_index(drop=True)
        lng_diff = data[column_name[1]].diff().dropna().reset_index(drop=True)

        # 確保格點和CSV數據的長度相同
        if len(diff_radar_grids) != len(data_diff):
            continue
        
        if len(data) < (window_size + step_size) + 1:
            # 資料不夠長，無法進行滑動窗口處理(至少需要 window_size + step_size + 1 筆資料)
            continue

        x, y_lat, y_lng = [], [], []
        for i in range(0, len(diff_radar_grids) - window_size - 1, 1):
            # 格點窗口
            x.append(diff_radar_grids[i:i + window_size])

            lat_d3 = lat_diff.values[i + window_size]
            lat_d4 = lat_diff.values[i + window_size + 1]
            lng_d3 = lng_diff.values[i + window_size]
            lng_d4 = lng_diff.values[i + window_size + 1]

            y_lat.append([lat_d3, lat_d4])  # [Δ₃, Δ₄]
            y_lng.append([lng_d3, lng_d4])  # [Δ₃, Δ₄]

        for x_sample, y_lat_sample, y_lng_sample in zip(x, y_lat, y_lng):
            yield np.array(x_sample, dtype=np.float32), {
                'convlstm_lat_output': np.array(y_lat_sample, dtype=np.float32),  # 緯度差標籤不縮放
                'convlstm_lng_output': np.array(y_lng_sample, dtype=np.float32)   # 經度差標籤不縮放
            }


def create_radar_csv_sliding_window_dataset(radar_files, csv_files, column_name, window_size, step_size, height, width, channels, mode='train'):
    """
    使用滑動窗口生成TensorFlow Dataset，處理格點作為X，並用CSV中的y作為標籤。
    """
    dataset = tf.data.Dataset.from_generator(
        lambda: radar_csv_sliding_window_generator(
            radar_files, csv_files, column_name, window_size, step_size, height, width, channels, mode),
        output_signature=(
            tf.TensorSpec(shape=(window_size, height, width,
                          channels), dtype=tf.float32),
            {
                'convlstm_lat_output': tf.TensorSpec(shape=(step_size,), dtype=tf.float32),  # 緯度標籤
                'convlstm_lng_output': tf.TensorSpec(shape=(step_size,), dtype=tf.float32)   # 經度標籤
            }
        )
    )
    return dataset

In [5]:
split_data_mode = 'month'  # 'month' or 'sequence'
# cells_csv_path = r'H:\cell_data_processed\cells'
# cells_radar_path = r'H:\cell_data_processed\radar\grids\global'
cells_csv_path = r'E:\YuCheng\cell_data_processed\cells'
cells_radar_path = r'E:\YuCheng\cell_data_processed\radar\grids\global'


if split_data_mode == 'month':
    train_radar_files, val_radar_files, test_radar_files = split_data_by_month(
        cells_radar_path)
    train_csv_files, val_csv_files, test_csv_files = get_cells_csv_files(
        cells_csv_path, train_radar_files=train_radar_files, val_radar_files=val_radar_files, test_radar_files=test_radar_files)
else:
    train_radar_files, val_radar_files, test_radar_files = split_data_by_sequence(
        cells_radar_path)
    train_csv_files, val_csv_files, test_csv_files = get_cells_csv_files(
        cells_csv_path, train_radar_files=train_radar_files, val_radar_files=val_radar_files, test_radar_files=test_radar_files)

# test_radar_files, test_csv_files = intersect_test_files(cells_radar_path, cells_csv_path)

print('train_radar_files:', len(train_radar_files))
print('val_radar_files:', len(val_radar_files))
print('test_radar_files:', len(test_radar_files))
print('---' * 10)
print('train_csv_files:', len(train_csv_files))
print('val_csv_files:', len(val_csv_files))
print('test_csv_files:', len(test_csv_files))

train_radar_files: 53023
val_radar_files: 13103
test_radar_files: 14647
------------------------------
train_csv_files: 53023
val_csv_files: 13103
test_csv_files: 14647


In [ ]:
column_name = ['Latitude', 'Longitude']
# 設定滑動窗口的參數
window_size = 5  # 窗口大小
step_size = 2  # 步長

height, width, channels = 224, 224, 1 # 格點大小為224x224, 格點

# 創建radar格點和CSV數據集
train_dataset = create_radar_csv_sliding_window_dataset(
    train_radar_files, train_csv_files, column_name, window_size, step_size, height, width, channels, mode='train')

val_dataset = create_radar_csv_sliding_window_dataset(
    val_radar_files, val_csv_files, column_name, window_size, step_size, height, width, channels, mode='val')

In [7]:
print(train_dataset.element_spec)
print(val_dataset.element_spec)

(TensorSpec(shape=(4, 224, 224, 1), dtype=tf.float32, name=None), {'convlstm_lat_output': TensorSpec(shape=(2,), dtype=tf.float32, name=None), 'convlstm_lng_output': TensorSpec(shape=(2,), dtype=tf.float32, name=None)})
(TensorSpec(shape=(4, 224, 224, 1), dtype=tf.float32, name=None), {'convlstm_lat_output': TensorSpec(shape=(2,), dtype=tf.float32, name=None), 'convlstm_lng_output': TensorSpec(shape=(2,), dtype=tf.float32, name=None)})


# 建立模型

In [ ]:
# 0117 加 GlobalAveragePooling2D, 並將第三個 ConvLSTM 層改為 return_sequences=False
early_stopping = EarlyStopping(monitor='val_loss', patience=5,
                               verbose=1, mode='auto', restore_best_weights=True)

checkpoint = ModelCheckpoint(os.path.join(
    os.getcwd(), 'weights', split_data_mode, 'non_cumulative_diff','convlstm_multitask', 'convlstm_mt_diff5-2_e{epoch:02d}v{val_loss:.4f}'),
    monitor='val_loss', save_best_only=True)

n_in = window_size
inputs = Input(shape=(n_in, height, width, channels), name='convlstm_input')

# 第一個 ConvLSTM 層
x = ConvLSTM2D(filters=32, kernel_size=(3, 3), activation=LeakyReLU(
), padding='same', return_sequences=True, name='convlstm_1')(inputs)
x = TimeDistributed(BatchNormalization(name='convlstm_1-bn_1'),
                    name='time-distributed_1-1')(x)
x = TimeDistributed(MaxPooling2D(pool_size=(
    2, 2), name='max-pooling_1'), name='time-distributed_1-2')(x)

# 第二個 ConvLSTM 層
x = ConvLSTM2D(filters=64, kernel_size=(5, 5), activation=LeakyReLU(
), padding='same', return_sequences=True, name='convlstm_2')(x)
x = TimeDistributed(BatchNormalization(name='convlstm_2-bn_2'),
                    name='time-distributed_2-1')(x)

# 第三個 ConvLSTM 層 
x = ConvLSTM2D(filters=128, kernel_size=(3, 3), activation='tanh', padding='same', 
               return_sequences=False, name='convlstm_3')(x)
x = BatchNormalization(name='convlstm_3-bn_3')(x)

# Global Average Pooling
x = GlobalAveragePooling2D(name='global_avg_pool')(x)

# 展平層
# x = Flatten(name='flatten')(x)

# 全連接層
x = Dense(64, activation='linear',
          kernel_regularizer=l2(0.01), name='dense_1')(x)
x = Dropout(0.2, name='convlstm_dense-dropout_1')(x)

x = Dense(32, activation='linear',
          kernel_regularizer=l2(0.01), name='dense_2')(x)
x = Dropout(0.1, name='convlstm_dense-dropout_2')(x)

# 分支1：緯度差預測
latitude_output = Dense(2, activation='linear', name='convlstm_lat_output')(x)

# 分支2：經度差預測
longitude_output = Dense(2, activation='linear', name='convlstm_lng_output')(x)

# 定義多任務模型
convlstm_model = Model(inputs=inputs, outputs=[
                       latitude_output, longitude_output], name='convlstm_multi_task')

# 編譯模型，使用自定義損失函數
convlstm_model.compile(optimizer=Adam(learning_rate=0.0001),
                       loss={'convlstm_lat_output': 'mse',
                             'convlstm_lng_output': 'mse'},
                       loss_weights={'convlstm_lat_output': 1.0,
                                     'convlstm_lng_output': 1.0},
                       metrics={'convlstm_lat_output': ['mse', rmse, 'mae'],
                                'convlstm_lng_output': ['mse', rmse, 'mae'],})                    

# 打印模型摘要
print(convlstm_model.summary())

Model: "convlstm_multi_task"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 convlstm_input (InputLayer)    [(None, 4, 224, 224  0           []                               
                                , 1)]                                                             
                                                                                                  
 convlstm_1 (ConvLSTM2D)        (None, 4, 224, 224,  38144       ['convlstm_input[0][0]']         
                                 32)                                                              
                                                                                                  
 time-distributed_1-1 (TimeDist  (None, 4, 224, 224,  128        ['convlstm_1[0][0]']             
 ributed)                        32)                                            

In [9]:
# 設定訓練參數
batch_size = 4
epochs = 50

# 使用 .batch() 和 .prefetch() 進行數據集的優化加載
train_dataset = train_dataset.batch(batch_size)\
                .prefetch(tf.data.experimental.AUTOTUNE)

val_dataset = val_dataset.batch(batch_size)\
                .prefetch(tf.data.experimental.AUTOTUNE)

In [10]:
# index = 0
# for x, y in train_dataset:  # 查看訓練集第一個批次的數據形狀
#     if x.shape[1:] != (2, 224, 224, 1):
#         print(f"Input shape: {x.shape}")
#         print(f"Index: {index}")
#         print('Input shape is not correct!')
#     index += 1
# print('train tatal', index)
    
# index = 0
# for x, y in val_dataset:  # 查看驗證集第一個批次的數據形狀
#     if x.shape[1:] != (2, 224, 224, 1):
#         print(f"Input shape: {x.shape}")
#         print(f"Index: {index}")
#         print('Input shape is not correct!')
#     index += 1
# print('val tatal', index)


In [11]:
# 開始訓練模型
history = convlstm_model.fit(
    train_dataset,
    validation_data=val_dataset,  # 傳入驗證集
    epochs=epochs,
    callbacks=[early_stopping, checkpoint],
    verbose=1  # 訓練過程中打印進度
)

Epoch 1/50
  17816/Unknown - 10633s 596ms/step - loss: 0.0575 - convlstm_lat_output_loss: 2.6905e-04 - convlstm_lng_output_loss: 3.8011e-04 - convlstm_lat_output_mse: 2.6905e-04 - convlstm_lat_output_rmse: 0.0136 - convlstm_lat_output_mae: 0.0114 - convlstm_lng_output_mse: 3.8011e-04 - convlstm_lng_output_rmse: 0.0160 - convlstm_lng_output_mae: 0.0136

INFO:tensorflow:Assets written to: e:\YuCheng\master_thesis\multi-step\weights\month\non_cumulative_diff\convlstm_multitask\convlstm_mt_diff4-2_e01v0.0012\assets


INFO:tensorflow:Assets written to: e:\YuCheng\master_thesis\multi-step\weights\month\non_cumulative_diff\convlstm_multitask\convlstm_mt_diff4-2_e01v0.0012\assets


17816/17816 [==============================] - 13976s 784ms/step - loss: 0.0575 - convlstm_lat_output_loss: 2.6905e-04 - convlstm_lng_output_loss: 3.8011e-04 - convlstm_lat_output_mse: 2.6905e-04 - convlstm_lat_output_rmse: 0.0136 - convlstm_lat_output_mae: 0.0114 - convlstm_lng_output_mse: 3.8011e-04 - convlstm_lng_output_rmse: 0.0160 - convlstm_lng_output_mae: 0.0136 - val_loss: 0.0012 - val_convlstm_lat_output_loss: 3.7540e-04 - val_convlstm_lng_output_loss: 7.5942e-04 - val_convlstm_lat_output_mse: 3.7540e-04 - val_convlstm_lat_output_rmse: 0.0166 - val_convlstm_lat_output_mae: 0.0145 - val_convlstm_lng_output_mse: 7.5942e-04 - val_convlstm_lng_output_rmse: 0.0241 - val_convlstm_lng_output_mae: 0.0223
Epoch 2/50
17816/17816 [==============================] - 16011s 899ms/step - loss: 5.3901e-04 - convlstm_lat_output_loss: 2.1328e-04 - convlstm_lng_output_loss: 2.9590e-04 - convlstm_lat_output_mse: 2.1328e-04 - convlstm_lat_output_rmse: 0.0129 - convlstm_lat_output_mae: 0.0109 - con

KeyboardInterrupt: 

# 評估模型

In [ ]:
column_name = ['Latitude', 'Longitude']
# 設定滑動窗口的參數
window_size = 5  # 窗口大小
step_size = 2  # 步長

height, width, channels = 224, 224, 1  # 格點大小為224x224, 格點

# 測試資料集
test_dataset = create_radar_csv_sliding_window_dataset(
    test_radar_files, test_csv_files, column_name, window_size, step_size, height, width, channels, mode='test')

In [9]:
test_dataset.element_spec

(TensorSpec(shape=(2, 224, 224, 1), dtype=tf.float32, name=None),
 {'convlstm_lat_output': TensorSpec(shape=(2,), dtype=tf.float32, name=None),
  'convlstm_lng_output': TensorSpec(shape=(2,), dtype=tf.float32, name=None)})

In [10]:
# 設定訓練參數
batch_size = 4
epochs = 50

# 使用 .batch() 和 .prefetch() 進行數據集的優化加載
test_dataset = test_dataset.batch(batch_size)\
                .prefetch(tf.data.experimental.AUTOTUNE)

In [11]:
if split_data_mode == 'month':
    model_path = os.path.join(os.getcwd(), r'weights\month\non_cumulative_diff\convlstm_multitask\convlstm_mt_diff2-2_e01v0.0014')
else:
    pass
    # model_path = os.path.join(os.getcwd(), r'weights\sequence\convlstm_multitask\convlstm_mt_diff2-1_e01v0.0018')


if os.path.exists(model_path):
    convlstm_model = load_model(model_path, custom_objects={'rmse': rmse})
    print('Load model successfully!')
    print(convlstm_model.summary())

Load model successfully!
Model: "convlstm_multi_task"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 convlstm_input (InputLayer)    [(None, 2, 224, 224  0           []                               
                                , 1)]                                                             
                                                                                                  
 convlstm_1 (ConvLSTM2D)        (None, 2, 224, 224,  38144       ['convlstm_input[0][0]']         
                                 32)                                                              
                                                                                                  
 time-distributed_1-1 (TimeDist  (None, 2, 224, 224,  128        ['convlstm_1[0][0]']             
 ributed)                        32)                   

In [12]:
# evaluation = convlstm_model.evaluate(test_dataset)

# print("=== 經緯度差（difference）評估 ===")

# print(f'Total Loss: {evaluation[0]:.5f}')         # 總損失

# print(f'Latitude Loss: {evaluation[1]:.5f}')       # 緯度 MSE
# print(f'Longitude Loss: {evaluation[2]:.5f}')      # 經度 MSE

# print(f'Latitude MSE: {evaluation[3]:.5f}')       # 緯度 MSE
# print(f'Longitude MSE: {evaluation[6]:.5f}')      # 經度 MSE

# print(f'Latitude RMSE: {evaluation[4]:.5f}')       # 緯度 RMSE
# print(f'Longitude RMSE: {evaluation[7]:.5f}')      # 經度 RMSE

# print(f'Latitude MAE: {evaluation[5]:.5f}')       # 緯度 MAE
# print(f'Longitude MAE: {evaluation[8]:.5f}')      # 經度 MAE

# import math

# # ============= 計算誤差(公里) =============
# def calculate_mae_distance(lat_mae: float, lon_mae: float, latitude: float = 25.071182):
#     lat_km = lat_mae * 111
#     lon_km = lon_mae * 111 * math.cos(math.radians(latitude))
#     return math.sqrt(lat_km**2 + lon_km**2)

# # 範例數據：經度差 MAE = 0.02，緯度差 MAE = 0.01，五分山雷達站的緯度 = 23.5
# lat_mae = evaluation[5]
# lon_mae = evaluation[8]
# # 固定緯度
# # 五分山雷達站的經緯度
# center_lat = 25.071182
# center_lon = 121.781205

# mae_distance = calculate_mae_distance(lat_mae, lon_mae, center_lat)
# print(f"Average distance error (via MAE): {mae_distance:.2f} km")

In [13]:
from haversine import haversine

distances = []
lat_errors = []
lng_errors = []

# 分開統計 t+1 / t+2 的誤差
lat_errors_1, lng_errors_1, dist_1 = [], [], []
lat_errors_2, lng_errors_2, dist_2 = [], [], []

for radar_folder_path, csv_file_path in zip(test_radar_files, test_csv_files):
    df = pd.read_csv(csv_file_path)

    if len(df) < (window_size + step_size) + 1:
        # 資料不夠長，無法進行滑動窗口處理(至少需要 window_size + step_size + 1 筆資料)
        continue
    
    # 取得經緯度真實值
    lats = df['Latitude'].values
    lngs = df['Longitude'].values

    # 計算差值 (差分)
    delta_lat = np.diff(lats)
    delta_lng = np.diff(lngs)

    combined_data = np.vstack([delta_lat, delta_lng]).T  # 經緯度差異合併

    radar_grids = []
    radar_grid_names = sorted(os.listdir(radar_folder_path))

    if len(radar_grid_names) != len(df):
        continue

    for radar_grid_name in radar_grid_names:
        radar_grid_path = os.path.join(radar_folder_path, radar_grid_name)
        radar_grid = np.load(radar_grid_path)
        radar_grid_resized = cv2.resize(radar_grid, (height, width))
        radar_grids.append(radar_grid_resized)

    diff_radar_grids = radar_grid_diff_processing(np.array(radar_grids))

    # 準備滑動窗口資料
    inputs = []
    for i in range(0, len(diff_radar_grids) - window_size - 1, 1):
        # input_sample = [diff_radar_grids[i], diff_radar_grids[i+1]]
        input_sample = diff_radar_grids[i : i + window_size]
        inputs.append(input_sample)

    inputs = np.array(inputs)

    # 模型預測
    pred_lats_diff, pred_lngs_diff = convlstm_model.predict(inputs)  # shape: (samples, 2)


    # 差值還原回經緯度
    pred_lats = []
    pred_lngs = []
    for i, (dlat, dlng) in enumerate(zip(pred_lats_diff, pred_lngs_diff)):
        base_lat = lats[i + window_size]
        base_lng = lngs[i + window_size]
        
        pred_lat1 = base_lat + dlat[0]
        pred_lat2 = pred_lat1 + dlat[1]
        pred_lng1 = base_lng + dlng[0]
        pred_lng2 = pred_lng1 + dlng[1]

        pred_lats.append([pred_lat1, pred_lat2])
        pred_lngs.append([pred_lng1, pred_lng2])

    # 計算實際誤差
    for i in range(len(pred_lats)):
        real_lat1 = lats[i + window_size + 1]
        real_lng1 = lngs[i + window_size + 1]
        real_lat2 = lats[i + window_size + 2]
        real_lng2 = lngs[i + window_size + 2]

        pred_lat1 = pred_lats[i][0]
        pred_lng1 = pred_lngs[i][0]
        pred_lat2 = pred_lats[i][1]
        pred_lng2 = pred_lngs[i][1]

        # 全體誤差
        lat_errors.extend([real_lat1 - pred_lat1, real_lat2 - pred_lat2])
        lng_errors.extend([real_lng1 - pred_lng1, real_lng2 - pred_lng2])
        distances.extend([
            haversine((real_lat1, real_lng1), (pred_lat1, pred_lng1)),
            haversine((real_lat2, real_lng2), (pred_lat2, pred_lng2)),
        ])

        # t+1
        lat_errors_1.append(real_lat1 - pred_lat1)
        lng_errors_1.append(real_lng1 - pred_lng1)
        dist_1.append(haversine((real_lat1, real_lng1), (pred_lat1, pred_lng1)))

        # t+2
        lat_errors_2.append(real_lat2 - pred_lat2)
        lng_errors_2.append(real_lng2 - pred_lng2)
        dist_2.append(haversine((real_lat2, real_lng2), (pred_lat2, pred_lng2)))

# ========= 全部預測誤差 =========
lat_mae_pos = np.mean(np.abs(lat_errors))
lng_mae_pos = np.mean(np.abs(lng_errors))
lat_mse_pos = np.mean(np.square(lat_errors))
lng_mse_pos = np.mean(np.square(lng_errors))
lat_rmse_pos = np.sqrt(lat_mse_pos)
lng_rmse_pos = np.sqrt(lng_mse_pos)
avg_distance = np.mean(distances)

print("=== 經緯度位置（還原後）總體評估 ===")
print(f'Latitude MSE: {lat_mse_pos:.6f} 度')
print(f'Longitude MSE: {lng_mse_pos:.6f} 度')
print(f'Latitude MAE: {lat_mae_pos:.6f} 度')
print(f'Longitude MAE: {lng_mae_pos:.6f} 度')
print(f'Latitude RMSE: {lat_rmse_pos:.6f}')
print(f'Longitude RMSE: {lng_rmse_pos:.6f}')
print(f'Average Haversine distance error: {avg_distance:.3f} km\n')

# ========= t+1 預測誤差 =========
lat1_mae = np.mean(np.abs(lat_errors_1))
lng1_mae = np.mean(np.abs(lng_errors_1))
lat1_mse = np.mean(np.square(lat_errors_1))
lng1_mse = np.mean(np.square(lng_errors_1))
lat1_rmse = np.sqrt(np.mean(np.square(lat_errors_1)))
lng1_rmse = np.sqrt(np.mean(np.square(lng_errors_1)))
dist1_avg = np.mean(dist_1)

print("=== 第一步預測 (t+1) 評估 ===")
print(f'Latitude MSE: {lat1_mse:.6f} 度')
print(f'Longitude MSE: {lng1_mse:.6f} 度')
print(f'Latitude MAE: {lat1_mae:.6f} 度')
print(f'Longitude MAE: {lng1_mae:.6f} 度')
print(f'Latitude RMSE: {lat1_rmse:.6f}')
print(f'Longitude RMSE: {lng1_rmse:.6f}')
print(f'Average Haversine distance error: {dist1_avg:.3f} km\n')

# ========= t+2 預測誤差 =========
lat2_mae = np.mean(np.abs(lat_errors_2))
lng2_mae = np.mean(np.abs(lng_errors_2))
lat2_mse = np.mean(np.square(lat_errors_2))
lng2_mse = np.mean(np.square(lng_errors_2))
lat2_rmse = np.sqrt(np.mean(np.square(lat_errors_2)))
lng2_rmse = np.sqrt(np.mean(np.square(lng_errors_2)))
dist2_avg = np.mean(dist_2)

print("=== 第二步預測 (t+2) 評估 ===")
print(f'Latitude MSE: {lat2_mse:.6f} 度')
print(f'Longitude MSE: {lng2_mse:.6f} 度')
print(f'Latitude MAE: {lat2_mae:.6f} 度')
print(f'Longitude MAE: {lng2_mae:.6f} 度')
print(f'Latitude RMSE: {lat2_rmse:.6f}')
print(f'Longitude RMSE: {lng2_rmse:.6f}')
print(f'Average Haversine distance error: {dist2_avg:.3f} km')

1/1 [==============================] - 0s 21ms/step
=== 經緯度位置（還原後）總體評估 ===
Latitude MSE: 0.000724 度
Longitude MSE: 0.002369 度
Latitude MAE: 0.019847 度
Longitude MAE: 0.038060 度
Latitude RMSE: 0.026901
Longitude RMSE: 0.048668
Average Haversine distance error: 4.791 km

=== 第一步預測 (t+1) 評估 ===
Latitude MSE: 0.000331 度
Longitude MSE: 0.001003 度
Latitude MAE: 0.013867 度
Longitude MAE: 0.026085 度
Latitude RMSE: 0.018205
Longitude RMSE: 0.031678
Average Haversine distance error: 3.306 km

=== 第二步預測 (t+2) 評估 ===
Latitude MSE: 0.001116 度
Longitude MSE: 0.003734 度
Latitude MAE: 0.025827 度
Longitude MAE: 0.050036 度
Latitude RMSE: 0.033405
Longitude RMSE: 0.061104
Average Haversine distance error: 6.277 km
